In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pymongo sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 13.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [ ]:
!pip install transformers -U
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 25.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.0 MB/s eta 0:00:00


In [ ]:
import pymongo

def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None


mongo_uri = "mongodb+srv://habiakl:***@***.fhfyoiv.mongodb.net/?retryWrites=true&w=majority&appName=Kerberos"

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client["semantic_tower"]
collection = db["geo"]

Connection to MongoDB successful


In [ ]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 660, 'electionId': ObjectId('7fffffff0000000000000008'), 'opTime': {'ts': Timestamp(1719650192, 281), 't': 8}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1719650192, 290), 'signature': {'hash': b'\xa9\xe8\\S\xe3\xc6\xb4\nMH\xb9`\x99N\x7f\x97\xa1\x8c E', 'keyId': 7359578748351414277}}, 'operationTime': Timestamp(1719650192, 281)}, acknowledged=True)

In [ ]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")


def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# load geo dataset from dataframe
import pandas as pd

dataset_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/llms4ol/geo/data/geo_semantic_primes.csv")
dataset_df = dataset_df.drop(columns=["embedding"]) # remove string embeddings to re-generate embeddings as vectors
dataset_df.loc[dataset_df['primes'].isnull(),'primes'] = dataset_df['entity']  # fill empty primes with entity class itself: this makes it a local universal prime
dataset_df["embedding"] = dataset_df["primes"].apply(get_embedding)

In [ ]:
# insert dataset into mongodb
documents = dataset_df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


In [ ]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 660,  # Number of candidate matches to consider
                "limit": 1,  # Return top 1 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "entity": 1,  # Include the entity field
                "primes": 1,  # Include the primes field
                "embedding": 1, # Include the embedding field
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
        },
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

In [ ]:
def get_search_result(query, collection):

    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += f"{result.get('entity', 'N/A')}"

    return search_result

In [ ]:
# get list of wordnet entities that need semantic universals
import json

with open("/content/drive/MyDrive/Colab Notebooks/llms4ol/geo/data/geonames_train.json", "r") as json_data:
    geonames_json = json.loads(json_data.read())
    json_data.close()

# get all wordnet entities
entities_list = [i['type'] for i in geonames_json]
# remove all duplicate entities
entities = list(set(entities_list))
len(entities)

660

In [ ]:
# Conduct query with retrival of sources
term = "Rhinos Casino"
#query = f'Using the Search Results, give the best entity for the term {term} from the list {entities}'
query = f'give the entity for the term {term}'
source_information = get_search_result(query, collection)
print("*** Source Information:", source_information)
combined_information = (
    f"Query: {query}\nSelect the answer from this list {entities} relying on the Search Results:\n{source_information}|"
)

print(combined_information)

*** Source Information: casino
Query: give the entity for the term Rhinos Casino
Select the answer from this list ['copper works', 'lake', 'gorge(s)', 'former sugar mill', 'maritime school', 'halting place', 'street', 'desert', 'dry stream bed', 'inlet', 'market', 'meadow', 'military school', 'hospital', 'brewery', 'plain(s)', 'wetland', 'fissure', 'moraine', 'saddle', 'communication center', 'pyramid', 'steps', 'foundry', 'slide', 'post office', 'dike', 'intermittent oxbow lake', 'inspection station', 'windmill', 'lost river', 'storehouse', 'fossilized forest', 'portage', 'oil pipeline terminal', 'ruined dam', 'weir(s)', 'clearing', 'rock desert', 'valley', 'church', 'military installation', 'terminal', 'waterhole(s)', 'railroad signal', 'sewage treatment plant', 'terrace', 'racetrack', 'oilfield', 'nunataks', 'amusement park', 'badlands', 'mesa(s)', 'railroad tunnel', 'ghat', 'baling station', 'estuary', 'spring(s)', 'wadies', 'stream', 'third-order administrative division', 'ruined 

In [ ]:
# authenticate to HF to be able to use some restricted LLMs
from huggingface_hub import notebook_login

notebook_login()

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, T5TokenizerFast, T5ForConditionalGeneration
"""
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# CPU Enabled uncomment below 👇🏽
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")
"""

model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Colab Notebooks/llms4ol/geo/model/flan-t5-small_geonames_model")
tokenizer = T5TokenizerFast.from_pretrained("/content/drive/MyDrive/Colab Notebooks/llms4ol/geo/model/flan-t5-small_geonames_tokenizer")
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
import torch

Q_LEN = 512   # Question Length
DEVICE = "cuda:0"

# test example
term = "Banana Bank"
query = f"""{term}"""
source_information = get_search_result(query, collection)
print("*** SOURCE:", source_information)
combined_information = (
    f"""{source_information}"""
    #f"""[CLS] {source_information} [SEP] {term}"""
    #f""""""
)
"""
# Moving tensors to GPU
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=500)
response_string = tokenizer.decode(response[0])
#print(tokenizer.decode(response[0]))
"""
inputs = tokenizer(term, combined_information, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

predicted_type = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)

"""
# initializing substrings
sub1 = "<pad> "
sub2 = "</s>"

# getting elements in between using split() and join()
answer = ''.join(response_string.split(sub1)[1].split(sub2)[0])
"""
# printing result
#print("The answer is :",answer)
print("The answer is :",predicted_type)
print(type(predicted_type))

*** SOURCE: section of bank
The answer is : bank
<class 'str'>


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
import numpy as np
import torch

Q_LEN = 512   # Question Length
DEVICE = "cuda:0"

def precision_at_k(actual, predicted):
    act_set = set(actual)
    pred_set = set(predicted)
    result = len(act_set & pred_set) / float(len(predicted))
    return result*100

def apk(actual, predicted, k):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if not actual:
        return 0.0
    if len(predicted)>k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    return score / min(len(actual), k)


def mapk(actual, predicted, k):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int,
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a, p, k) for a,p in zip(actual, predicted)])


class EvaluationMetrics:

    def __init__(self, ks:list, metric="map") -> None:
        self.ks = ks
        self.metric=metric

    def evaluate(self, actual:list, predicted:list):
        if self.metric == "map":
            return self.MAP(actual, predicted)
        else:
            return self.AP(actual, predicted)

    def MAP(self, actual:list, predicted:list):
        results_dict = {}
        for k in self.ks:
            results_dict["MAP@"+str(k)] = mapk(actual=actual, predicted=predicted, k=k)
        return results_dict

    def AP(self, actual:list, predicted:list):
        results_dict = {}
        for k in self.ks:
            results_dict["AP@"+str(k)] = [apk(actual=actual, predicted=predicted, k=k)
                                          for a, p in zip(actual, predicted)]
        return results_dict

def predict_answer(term, combined_information, ref_type=None):

    inputs = tokenizer(term, combined_information, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)
    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)
    predicted_type = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_type:
        # Load the MAP@K metric for K=1
        score = mapk([ref_type], [predicted_type], 1)

        print("Term: \n", term)
        return {
            "Reference Type: ": ref_type,
            "Predicted Type: ": predicted_type,
            "MAP@1: ": score
        }
    else:
        return {
            "Predicted Type: ": predicted_type
            }

In [ ]:
# prepare final submission
submission = []

# load test set for final submission
with open("/content/drive/MyDrive/Colab Notebooks/llms4ol/geo/data/A.2(FS)_GeoNames_Test.json", "r") as json_data:
    geo_test_json = json.loads(json_data.read())
    json_data.close()

# test predictions
for row in geo_test_json:
  termid = row["ID"]
  term = row["term"]
  print("*** TERM ***")
  print(term)
  query = f"""{term}"""
  source_information = get_search_result(query, collection)
  combined_information = (
      f"""{source_information}"""
  )
  predicted_type = predict_answer(term, combined_information)["Predicted Type: "]
  # construct result
  result_dict = {}
  result_dict["ID"] = termid
  result_dict["type"] = []
  result_dict["type"].append(predicted_type)
  # add result to final submission
  submission.append(result_dict)

# write final submission to JSON file to adhere to submission format
with open("A.1(FS)_Geo_submission.json", "w") as final:
    json.dump(submission, final)

print("************* SUBMISSION FILE GENERATION SUCCESSFUL *************")

*** TERM ***
Coll de Vista


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
*** TERM ***
Estancia La Raquelita
*** TERM ***
Fabrica La Ramada
*** TERM ***
Rio Lara
*** TERM ***
Arroyo La Quesera
*** TERM ***
Puesto La Puntilla
*** TERM ***
Estancia La Punta
*** TERM ***
La Puerta Verde
*** TERM ***
La Puente
*** TERM ***
La Promesa
*** TERM ***
Cordon de La Potrerillos
*** TERM ***
Estancia La Positiva
*** TERM ***
La Poma
*** TERM ***
Cerro de La Polcura
*** TERM ***
La Playosa
*** TERM ***
Estancia La Plata
*** TERM ***
Mogote La Pintada
*** TERM ***
La Perseverancia
*** TERM ***
La Penas
*** TERM ***
La Pega
*** TERM ***
Estancia La Pava
*** TERM ***
La Pastora
*** TERM ***
Cerro La Parva
*** TERM ***
La Paquita
*** TERM ***
Estancia La Panchita
*** TERM ***
Estero La Paloma
*** TERM ***
La Palmita
*** TERM ***
La Palmera
*** TERM ***
La Palma
*** TERM ***
Isla La Paciencia
*** TERM ***
Arroyo Lapachal
*** TERM ***
La Otomana
*** TERM ***
La Osamenta
*** TERM ***
Arroyo La Ortiguita
*** TERM ***
La Ofelia
*

KeyboardInterrupt: 